# Laboratories

AOC Day 7 Part 1

This feels like a mixture of matrices and trees/graphs. Let's bring in our `Matrix` and `Neighbours` implementations from Day 4. I have updated it a bit to fit the data types in the current problem.

In [63]:
class Neighbours:
    def __init__(self, a: str | None, b: str | None, c: str | None, m: str | None, n: str | None, p: str | None, q: str | None, r: str | None) -> None:
        self.a = a
        self.b = b
        self.c = c
        self.m = m
        self.n = n
        self.p = p
        self.q = q
        self.r = r

    # This function returns what's immediately above the current node
    # i.e., the North direction.
    @staticmethod
    def parent(matrix: "Matrix", of: tuple[int, int]) -> str | None:
        return Neighbours.of(matrix=matrix, node=of).b

    @staticmethod
    def of(matrix: "Matrix", node: tuple[int, int]):
        # x = rows, y = cols
        (x, y) = node
        x_min = True if x - 1 < 0 else False
        y_min = True if y - 1 < 0 else False

        x_max = True if x + 1 >= matrix.num_rows() else False
        y_max = True if y + 1 >= matrix.num_cols() else False

        a = None if (x_min or y_min) else matrix[x-1, y-1]
        b = None if x_min else matrix[x -1, y]
        c = None if (x_min or y_max) else matrix[x - 1, y + 1]
        m = None if y_min else matrix[x, y - 1]
        p = None if (y_min or x_max) else matrix[x + 1, y - 1]
        n = None if y_max else matrix[x, y + 1]
        r = None if x_max else matrix[x + 1, y]
        q = None if (y_max or x_max) else matrix[x + 1, y + 1]

        return Neighbours(
            a, b, c, m, n, p, q, r
        )
    
    def to_dict(self):
        return {
            "a": self.a,
            "b": self.b,
            "c": self.c,
            "m": self.m,
            "n": self.n,
            "p": self.p,
            "q": self.q,
            "r": self.r
        }
    
    def __iter__(self):
        yield from [
            self.a,
            self.b,
            self.c,
            self.m,
            self.n,
            self.p,
            self.r,
            self.q
        ]
        
class Matrix:
    def __init__(self, rows: list[str]) -> None:
        self.current_idx = -1
        self.last_idx = len(rows)
        self._vec: list[list] = []
        self._prepare_matrix(rows)
        pass

    def __iter__(self):
        return self
    
    def __next__(self):
        self.current_idx += 1
        if self.current_idx < self.last_idx:
            return self._vec[self.current_idx]

    def _prepare_matrix(self, rows: list[str]) -> None:
        self._vec = [["S" if ch == "S" else  "^" if ch == "^" else "." for ch in line] for line in rows]
        
    def _neighbours(self, node: tuple[int, int]):
        return Neighbours.of(
            matrix=self,
            node=node
        )
    
    def __getitem__(self, key: tuple[int, int]):
        """
        key: (row, column)
        """
        (row, col) = key
        return self._vec[row][col]

    def __setitem__(self, key: tuple[int, int], value: str):
        """
        key: (row, column)
        """
        (row, col) = key
        self._vec[row][col] = value

    def __len__(self):
        return len(self._vec)
    
    def row(self, idx: int) -> list[str]:
        return self._vec[idx]

    def num_rows(self) -> int:
        return len(self._vec)
    
    def num_cols(self) -> int:
        return len(self._vec[0])
    
    def to_string(self) -> str:
        return str(self._vec)
    
    def pretty_print(self) -> None:
        for row in self._vec:
            print(''.join(row))
    
    def dim(self) -> tuple[int, int]:
        return (self.num_rows(), self.num_cols())
    
    def apply_row(self, n, f):
        # Apply function f to an entire row.
        N = self._vec[n]
        for i in range(len(N)):
            f(self, N[i])
    
    def apply(self, f, row_major=True):
        if row_major:
            for i in range(len(self._vec)):
                for j in range(len(self._vec[0])):
                    f(self, self[i, j])
        else:
            for i in range(len(self._vec)):
                for j in range(len(self._vec[0])):
                    f(self, self[j, i])

    def save(self, filename: str):
        """
        Save the matrix to a file, one row per line.
        Each row is joined into a string.
        """
        with open(filename, 'w') as f:
            for row in self._vec:
                f.write(''.join(row) + '\n')


Given below is the input grid.

```
.......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
```

Given below is the path of the tachyon beam.

```
.......S.......
.......|.......
......|^|......
......|.|......
.....|^|^|.....
.....|.|.|.....
....|^|^|^|....
....|.|.|.|....
...|^|^|||^|...
...|.|.|||.|...
..|^|^|||^|^|..
..|.|.|||.|.|..
.|^|||^||.||^|.
.|.|||.||.||.|.
|^|^|^|^|^|||^|
|.|.|.|.|.|||.|
```

We need to model the behaviour of the tachyon beam first in order to solve this problem.

Let us read the input first.



In [64]:
input = [
    ".......S.......",
    "...............",
    ".......^.......",
    "...............",
    "......^.^......",
    "...............",
    ".....^.^.^.....",
    "...............",
    "....^.^...^....",
    "...............",
    "...^.^...^.^...",
    "...............",
    "..^...^.....^..",
    "...............",
    ".^.^.^.^.^...^.",
    "..............."
]

If we use our `Matrix` implementation to read this into a matrix:

In [65]:
mtx = Matrix(rows=input)

In [66]:
mtx.pretty_print()

.......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............


In [67]:
mtx.dim()

(16, 15)

Looks like the matrix implementation is okay.

We will add some utility functions into the `Matrix` class to traverse through and navigate the grid.

These functions are currently global for testing, but should be moved into the Matrix class after validation. Utility functions for beam navigation in the grid.

In [68]:
def initiate_split(matrix: Matrix):
    """
    Initiate the split from the starting point S.
    
    Note: This mutates the matrix.
    """
    first_row = matrix.row(0)
    start_node_idx = first_row.index("S")
    matrix[1, start_node_idx] = "|"

def quantum_split(matrix: Matrix, row: int):
    """Placeholder for logic to split the beam on the given row in the matrix. n here is the row index."""
    N = matrix.row(row)
    for i in range(len(N)):
        # TODO: We need to check if the beam has actually arrived here.
        if row - 1 >= 0 and matrix[row - 1, i] == "|" and N[i] == "^":
            # split the beam.
            if i - 1 >= 0:
                matrix[row, i - 1] = "|"
            if i + 1 < len(N):
                matrix[row, i + 1] = "|"
        elif row - 1 >= 0 and matrix[row - 1, i] == "|":
            matrix[row, i] = "|"

# Initiate the split.
matrix_cpy = Matrix(input)
initiate_split(matrix_cpy)

for r in range(matrix_cpy.num_rows()):
    quantum_split(matrix=matrix_cpy, row=r)
matrix_cpy.pretty_print()

.......S.......
.......|.......
......|^|......
......|.|......
.....|^|^|.....
.....|.|.|.....
....|^|^|^|....
....|.|.|.|....
...|^|^|||^|...
...|.|.|||.|...
..|^|^|||^|^|..
..|.|.|||.|.|..
.|^|||^||.||^|.
.|.|||.||.||.|.
|^|^|^|^|^|||^|
|.|.|.|.|.|||.|


Looks like we are good!

Let's update the `Matrix` class to our final implementation.

In [69]:
class Matrix:
    def __init__(self, rows: list[str]) -> None:
        self.current_idx = -1
        self.last_idx = len(rows)
        self._vec: list[list] = []
        self._prepare_matrix(rows)
        self._initiate_split()
        self.split_count = 0

    def __iter__(self):
        return self
    
    def __next__(self):
        self.current_idx += 1
        if self.current_idx < self.last_idx:
            return self._vec[self.current_idx]

    def _prepare_matrix(self, rows: list[str]) -> None:
        self._vec = [["S" if ch == "S" else  "^" if ch == "^" else "." for ch in line] for line in rows]

    def __getitem__(self, key: tuple[int, int]):
        """
        key: (row, column)
        """
        (row, col) = key
        return self._vec[row][col]

    def __setitem__(self, key: tuple[int, int], value: str):
        """
        key: (row, column)
        """
        (row, col) = key
        self._vec[row][col] = value

    def __len__(self):
        return len(self._vec)
    
    def _initiate_split(self):
        """
        Initiate the split from the starting point S.
        
        Note: This mutates the matrix.
        """
        first_row = self.row(0)
        start_node_idx = first_row.index("S")
        self[1, start_node_idx] = "|"

    def split_beam(self):
        """
        Splits the beam.
        """
        for row in range(self.num_rows()):
            N = self.row(row)
            for i in range(len(N)):
                if row - 1 >= 0 and self[row - 1, i] == "|" and N[i] == "^":
                    # split the beam.
                    if i - 1 >= 0:
                        self[row, i - 1] = "|"
                    if i + 1 < len(N):
                        self[row, i + 1] = "|"
                    self.split_count += 1
                elif row - 1 >= 0 and self[row - 1, i] == "|":
                    self[row, i] = "|"
    
    def row(self, idx: int) -> list[str]:
        return self._vec[idx]

    def num_rows(self) -> int:
        return len(self._vec)
    
    def num_cols(self) -> int:
        return len(self._vec[0])
    
    def to_string(self) -> str:
        return str(self._vec)
    
    def pretty_print(self) -> None:
        for row in self._vec:
            print(''.join(row))
    
    def dim(self) -> tuple[int, int]:
        return (self.num_rows(), self.num_cols())
    
    def apply_row(self, n, f):
        # Apply function f to an entire row.
        N = self._vec[n]
        for i in range(len(N)):
            f(self, N[i])
    
    def apply(self, f, row_major=True):
        if row_major:
            for i in range(len(self._vec)):
                for j in range(len(self._vec[0])):
                    f(self, self[i, j])
        else:
            for i in range(len(self._vec)):
                for j in range(len(self._vec[0])):
                    f(self, self[j, i])

    def save(self, filename: str):
        """
        Save the matrix to a file, one row per line.
        Each row is joined into a string.
        """
        with open(filename, 'w') as f:
            for row in self._vec:
                f.write(''.join(row) + '\n')

mtx = Matrix(input)
mtx.split_beam()
mtx.pretty_print()

.......S.......
.......|.......
......|^|......
......|.|......
.....|^|^|.....
.....|.|.|.....
....|^|^|^|....
....|.|.|.|....
...|^|^|||^|...
...|.|.|||.|...
..|^|^|||^|^|..
..|.|.|||.|.|..
.|^|||^||.||^|.
.|.|||.||.||.|.
|^|^|^|^|^|||^|
|.|.|.|.|.|||.|


We are good with the `Matrix` class implementation.

In [70]:
mtx.split_count

21

$21$ is in fact, the correct answer!

Let's try to perform the function for our input function.

In [71]:
with open(file="input.txt") as file:
    input = [line.rstrip() for line in file]

    mtx = Matrix(rows=input)
    mtx.split_beam()
    print(f"Split count = {mtx.split_count}")

    mtx.save("tree.txt")

Split count = 1579


$1579$ is the answer!

---

For part 2, we will have to re-define the splitting behaviour.

In [72]:
def initiate_split(matrix: Matrix):
    """
    Initiate the split from the starting point S.
    
    Note: This mutates the matrix.
    """
    first_row = matrix.row(0)
    start_node_idx = first_row.index("S")
    matrix[1, start_node_idx] = "|"

def quantum_split(matrix: Matrix, row: int):
    """Placeholder for logic to split the beam on the given row in the matrix. n here is the row index."""
    N = matrix.row(row)
    for i in range(len(N)):
        if row - 1 >= 0 and matrix[row - 1, i] == "|" and N[i] == "^":
            # split the beam.
            if i - 1 >= 0:
                # The left branch starts here.
                matrix[row, i - 1] = "|"
                pass
            if i + 1 < len(N):
                # The right branch starts here.
                matrix[row, i + 1] = "|"
                pass
            pass
        elif row - 1 >= 0 and matrix[row - 1, i] == "|":
            matrix[row, i] = "|"

If we can model the splitting behaviour into a **tree**, we could do a Depth-First Search (DFS) and come to the answer.

Let us try to create a `Graph` class having `Node` nodes.

In [73]:
from collections import defaultdict

class Node:
    def __init__(self, coordinate: tuple[int, int], value: str):
        self.value = value
        self.x, self.y = coordinate

    def __eq__(self, value: "Node") -> bool:
        return (self.x, self.y) == (value.x, value.y)

class Graph:
    def __init__(self):
        self.graph = defaultdict(list)

    def add_edge(self, u: Node, v: Node):
        self.graph[u].append(v)

    def _dfs(self, v: Node, visited: set):
        visited.add(v)

        for nbr in self.graph[v]:
            if nbr not in visited:
                self._dfs(nbr, visited)

    def dfs(self, v: Node):
        visited = set()
        self._dfs(v, visited)

In [ ]:
input = [
    ".......S.......",
    "...............",
    ".......^.......",
    "...............",
    "......^.^......",
    "...............",
    ".....^.^.^.....",
    "...............",
    "....^.^...^....",
    "...............",
    "...^.^...^.^...",
    "...............",
    "..^...^.....^..",
    "...............",
    ".^.^.^.^.^...^.",
    "..............."
]

matrix = Matrix(rows=input)
matrix.split_beam()
matrix.pretty_print()

def quantum_split(mtx: Matrix):
    """
    Splits the beam.
    """
    # splitters = defaultdict(list)
    splitters = [] # This is a temporary list of coordinates.

    for row in range(mtx.num_rows()):
        current_row = mtx.row(row)
        for i in range(len(current_row)):
            # Current value could also be accessed via matrix[row, i].
            current_value = current_row[i]
            current_node = (row, i)
            # If the parent node value is a beam, and the current node value is a splitter:
            if row - 1 >= 0 and mtx[row - 1, i] == "|" and current_value == "^":
                # Assume by this time the beam has already been split.

                # we need to find the previous splitter.
                splitters.append(current_node)

                # Now, we need to somehow add the edges.

    print(splitters)
                

quantum_split(matrix)

.......S.......
.......|.......
......|^|......
......|.|......
.....|^|^|.....
.....|.|.|.....
....|^|^|^|....
....|.|.|.|....
...|^|^|||^|...
...|.|.|||.|...
..|^|^|||^|^|..
..|.|.|||.|.|..
.|^|||^||.||^|.
.|.|||.||.||.|.
|^|^|^|^|^|||^|
|.|.|.|.|.|||.|
[(2, 7), (4, 6), (4, 8), (6, 5), (6, 7), (6, 9), (8, 4), (8, 6), (8, 10), (10, 3), (10, 5), (10, 9), (10, 11), (12, 2), (12, 6), (12, 12), (14, 1), (14, 3), (14, 5), (14, 7), (14, 13)]


We need to identify the edges and nodes here.

If we consider to create a `Node` for each `row - 1 >= 0 and matrix[row - 1, i] == "|" and current_value == "^"` call, that would mean we would create a node for each grid coordinate $(i,j)$ that the beam enters into.

Note that 

```python
row - 1 >= 0 and matrix[row - 1, i] == "|" and current_value == "^"
```

statement checks if the current node is a splitter, and the parent for the splitter is a beam.

**Idea**: We _could_ backtrack along the beam starting from a splitter node, and check if the right node of each beam node is a splitter. That splitter, in theory, is the parent of the current splitter node.